In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import numpy as np
import pandas as pd
# Plot Images
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
plt.style.use('ggplot')
import matplotlib
# Set GPU usage
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# Plotly 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
from studio.evaluation.keras import metrics, utils, visualizer
from studio.evaluation.keras.evaluators import CNNEvaluator
from studio.evaluation.keras.evaluators import Evaluator
from studio.evaluation.keras.evaluators import SequentialCNNEvaluator
from studio.evaluation.keras.evaluators import VisualQAEvaluator
from visual_qa.visual_qa import VisualQA

In [ ]:
# Example paths
report_dir = '../report/'
model_path = '/data/models/example_model/model_max_acc_1gpu.h5'
visual_dictionary = 'qa-data/data/visual_classifier/visual_dictionary.json'
by_definition_csv = 'qa-data/data/by-definition/by_definition_matrix.csv'
qa_data_json = 'qa-data/data/testset.json'
valid_evidence = 'qa-data/data/valid_evidence.json'
visual_data_dir = '/data/datasets/visual_qa/test_set'

In [ ]:
# Visual Object
cnn_evaluator = CNNEvaluator(
        concept_dictionary_path=None,
        custom_objects=None,
        concepts=None,
        model_path=model_path,
        batch_size=32,
        verbose=0)

In [ ]:
# Visual QA Object
visual_qa_evaluator = VisualQAEvaluator(
    report_dir=report_dir,
    qa_data_json = qa_data_json,
    visual_dictionary = visual_dictionary,
    by_definition_csv = by_definition_csv,
    valid_evidence = valid_evidence
)

In [ ]:
filtered_testset = visual_qa_evaluator.filtered_qa_data
filtered_diagnosis_ids = list(visual_qa_evaluator.visual_qa.visual_diagnosis_ids)

## Get Visual results

In [ ]:
labels = []
image_paths = []
for sample in filtered_testset:
    image_paths.append(os.path.join(visual_data_dir, sample["image_id"]))
    labels.append(filtered_diagnosis_ids.index(sample['diagnosis_id']))
    
probs = cnn_evaluator.predict(image_paths)
labels = np.array(labels)

In [ ]:
visual_result = cnn_evaluator.get_metrics(probs, labels, top_k=5, concept_labels=filtered_diagnosis_ids)

## Evaluate By-definition filter and get metrics

In [ ]:
visual_qa_result = visual_qa_evaluator.evaluate(probs, differential=True, report=True)

In [ ]:
visual_qa_evaluator.plot_top_k_accuracy()

In [ ]:
utils.compare_visual_by_definition_results(visual_result['average']['accuracy'] ,visual_qa_result['average']['accuracy'] )

In [ ]:
visual_qa_evaluator.find_top_k_questions()